In [1]:
from collections.abc import MutableMapping
from map_base import MapBase
from unsorted_table_map import UnsortedTableMap
from ch7_source_code import PositionalList

# Reinforcement

## R-10.1
Give a concrete implementation of the `pop` method in the context of the `MutableMapping` class, relying only on the five primary abstract methods of that class.

Solution: The five primary abstract methods of `MutableMapping` class are:
1. `__delitem__`
2. `__getitem__`
3. `__iter__`
4. `__len__`
5. `__setitem__`

In [2]:
class MutMap(UnsortedTableMap): # UnsortedTableMap has overriden all abstract methods of MutableMap
    def pop(self, k):
        """Remove and return (k,v) pair (or raise KeyError)."""
        temp = self[k]
        del self[k]     # uses __delitem__ abstract method
        return (k, temp)

In [3]:
map = MutMap()
for i in [('a', 3), ('b', 5), ('c', 7)]:
        map[i[0]] = i[1]
print(list(map.items()))
print(map.pop('b'))
print(list(map.items()))

[('a', 3), ('b', 5), ('c', 7)]
('b', 5)
[('a', 3), ('c', 7)]


## R-10.2
Give a concrete implementation of the `items()` method in the context of the `MutableMapping` class, relying only on the five primary abstract methods of that class. What would its running time be if directly applied to the `UnsortedTableMap` subclass?

In [4]:
class MutMap(UnsortedTableMap):
    def items(self):    # items method already exits in MutableMapping, we are overriding it
        for k in self:      # uses __iter__ method
            yield (k, self[k])      # uses __getitem__method

In [5]:
map = MutMap()
for i in [('A', 1), ('B', 2), ('C', 3), ('D', 4), ('E', 5)]:
        map[i[0]] = i[1]
print(list(map.items()))

[('A', 1), ('B', 2), ('C', 3), ('D', 4), ('E', 5)]


If we apply `items()` method to `UnsortedTableMap` subclass, as done above, it will take $ O(n) $.

## R-10.3
Give a concrete implementation of the `items()` method directly within the `UnsortedTableMap` class, ensuring that the entire iteration runs in $ O(n) $ time.

Solution: Similar to previous question.

## R-10.4
What is the worst-case running time for inserting $ n $ key-value pairs into an initially empty map $ M $ that is implemented with the `UnsortedTableMap` class?

Solution: Each insertion into `UnsortedTableMap` takes following steps:
1. Search through all keys to find find matching key to one being inserted. $ O(n^2) $ by sum of A.P
2. If found the key, then update the key. $ O(1) $
3. If key is not found, append the (k,v) to table (self._table, a python list). $ O(1) $ amortized  

Hence, the worst-case running time is $ O(n^2) $.  
The best case is when we are inserting $ (k,v) $ pairs such that key is same everytime. It means we are updating same key over and over. This way there is single element in the map, and step 1 becomes $ O(1) $ for each insertion. For $ n $ insertions, we simply have $ O(n) $.

## R-10.5
Reimplement the `UnsortedTableMap` class from Section 10.1.5, using the `PositionalList` class from Section 7.4 rather than a Python list.

In [6]:
class UnsortedTableMap(MapBase):

    def __init__(self):
        self._table = PositionalList()

    def __getitem__(self, k):
        for item in self._table: # PL supports iteration, yields the elements (here _Item instances)
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))

    def __setitem__(self, k, v):
        for item in self._table:
            if k == item._key:
                item._value = v
                return
        # did not find match for key
        self._table.add_last(self._Item(k,v))   # add _Item objects to PL, similar to append

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        node = self._table.first()
        while node is not None:
            if node.element()._key == k:
                break
            node = self._table.after(node)
        if node is not None:
            self._table.delete(node)
            return
        raise KeyError('Key Error: ' + repr(k))

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key

In [7]:
positionalmap = UnsortedTableMap()
for i in [('A', 1), ('B', 2), ('C', 3), ('D', 4), ('E', 5)]:
        positionalmap[i[0]] = i[1]              # __setitem__ working
print(list(positionalmap.items()))
print(len(positionalmap))                       # __len__ working

print(positionalmap['C'])                       # __getitem__ working

del positionalmap['C']                          # __delitem__ working
print(list(positionalmap.items()))

print(len(positionalmap))

for i in positionalmap:                         # __iter__ working
    print(i)    

# del positionalmap['X']        raises KeyError as expected

[('A', 1), ('B', 2), ('C', 3), ('D', 4), ('E', 5)]
5
3
[('A', 1), ('B', 2), ('D', 4), ('E', 5)]
4
A
B
D
E


## R-10.6
Which of the hash table collision-handling schemes could tolerate a load factor above $ 1 $ and which could not?

$ \lambda = \frac{n}{N} $ is known as load factor, where $ n $ is the size of map and $ N $ is the size of underlying bucket array (a python list, or positional list).  
In separate chaining, each bucket has a secondary container. That's why even if we have more number of elements in map than the size of bucket array, we can handle the collision. The downside of having $ \lambda $ greater than $ 1 $ is that we cannot expect $ O(1) $ in several operations in hash-table.  

In open addressing, we don't use secondary container, so value of $ \lambda $ cannot be greater than $ 1 $.

## R-10.7
Our `Position` classes for lists and trees support the `__eq__` method so that two distinct position instances are considered equivalent if they refer to the same underlying node in a structure. For positions to be allowed as keys in a hash table, there must be a definition for the `__hash__` method that is consistent with this notion of equivalence. Provide such a `__hash__` method.


In [8]:
# changes that need to be made to __hash__ method in HashMapBase class
def __hash__(self, position):
    # find the key that has to be hashed
    item = position.element()  # here element at position is _Item object
    key = item._key
    return hash(key)

# changes that need to be made in Position nested class in PositionalList class
def __eq__(self, other):
    return type(other) is type(self) and other._node is self._node and hash(self) == hash(other)

If two positions are equal, then their hashes should also be equal.  
Example if $ x == y $, then $ hash(x) == hash(y) $

In above code, if we two positions are equivalent then their hash should also be equal.

## R-10.8
What would be a good hash code for a vehicle identification number that is a string of numbers and letters of the form "$ 9X9XX99X9XX999999 $", where a "$ 9 $" represents a digit and an "$ X $" represents a letter?

In [9]:
string = "9X9XX99X9XX999999"  # a string is unmutable so hashable
print(hash(string))

-1120639965579104668


## R-10.9
Draw the 11-entry hash table that results from using the hash function, $ h(i) = (3i+5) mod 11 $, to hash the keys $ 12, 44, 13, 88, 23, 94, 11, 39, 20, 16,$  and $ 5 $, assuming collisions are handled by chaining.

In [10]:
def find_index(n):
    index = (3*n + 5) % 11
    return index

# bucket array lst
lst = [None]*11     # N = 11 given in hashing function
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = []     # secondary container for collision handling in separate chaining
    lst[index].append(i)
print(lst)

[[13], [94, 39], None, None, None, [44, 88, 11], None, None, [12, 23], [16, 5], [20]]


## R-10.10
What is the result of the previous exercise, assuming collisions are handled by linear probing?

In [11]:
def find_index(n):
    index = (3*n + 5) % 11
    return index

# bucket array lst
N = 11          # load factor = 1
lst = [None] * N
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = i
    else:
        temp = index
        while lst[index] is not None:   # linear probing in action
            index = (temp + 1) % N
            temp = temp + 1
        lst[index] = i
print(lst)

N = 22          # load factor = 0.5
lst = [None] * N
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = i
    else:
        temp = index
        while lst[index] is not None:   # linear probing in action
            index = (temp + 1) % N
            temp = temp + 1
        lst[index] = i
print(lst)


[13, 94, 39, 16, 5, 44, 88, 11, 12, 23, 20]
[13, 94, 39, None, None, 44, 88, 11, 12, 23, 20, 16, 5, None, None, None, None, None, None, None, None, None]


## R-10.11
Show the result of Exercise R-10.9, assuming collisions are handled by quadratic probing, up to the point where the method fails.

Solution: $ index = h(k) $  
Linear probing iteratively tries $ BucketArray[index+1] $ until finding an empty bucket.  
Quadratic probing iteratively tries $ BucketArray[(index + f(i))\ mod\ N] $ for $ i = 0, 1, 2... $ where $ f(i) = i^2 $.

In [12]:
def find_index(n):
    index = (3*n + 5) % 11
    return index

# bucket array lst
N = 22              # when N = 11, load factor is 1, program is taking long time
lst = [None] * N
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = i
    else:
        j = 1
        temp = index
        while lst[index] is not None:   # quadratic probing in action
            index = (temp + j**2) % N
            j += 1
        lst[index] = i
print(lst)

[13, 94, 39, None, None, 44, 88, None, 12, 23, 20, None, None, 16, 11, None, None, None, 5, None, None, None]


## R-10.12
What is the result of Exercise R-10.9 when collisions are handled by double hashing using the secondary hash function $ h'(k) = 7 − (k\ mod\ 7) $ ?

Solution: $ index = h(k) $. This index is obtained by usign primary hashing function $ h(k) $.  
Sometimes double hashing is used to avoid the clustering by linear or quadratic probing.  
Double hashing iteratively tries $ BucketArray[(index + f(i))\ mod\ N] $ for $ i = 1, 2, 3 $ where $ f(i) = i.h'(k) $.

In [13]:
def find_index(n):
    index = (3*n + 5) % 11
    return index

# bucket array lst
N = 11          # load factor = 1
lst = [None] * N
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = i
    else:
        j = 1
        temp = index
        while lst[index] is not None:
            index = (temp + j*(7 - (i%7))) % N      # using secondary hashing function
            j += 1
        lst[index] = i
print(lst)

N = 22          # load factor = 0.5
lst = [None] * N
for i in [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = i
    else:
        j = 1
        temp = index
        while lst[index] is not None:
            index = (temp + j*(7 - (i%7))) % N      # using secondary hashing function
            j += 1
        lst[index] = i
print(lst)

[13, 94, 23, 88, 39, 44, 11, 5, 12, 16, 20]
[13, 94, None, None, 39, 44, None, None, 12, 16, 20, 88, None, 23, 11, 5, None, None, None, None, None, None]


## R-10.13
What is the worst-case time for putting $ n $ entries in an initially empty hash table, with collisions resolved by chaining? What is the best case?

Solution: The worst case time will be when all items are inserted in same bucket, ie. all items collide at same place. If an UnsortedTableMap is used as a bucket. Then we have already seen in R-10.4 that it takes $ O(n^2) $ in worst case to place $ n $ items in the map.  
The best case is when collision doesn't happen at all. We add items only at empty locations (None in clase UnsortedTableMap) or if the location is not empty, then key at that location is same as key we are inserting, both of which take $ O(1) $ for every insertion, and $ O(n) $ for $ n $ insertions.

## R-10.14
Show the result of rehashing the hash table shown in Figure 10.6 into a table of size $ 19 $ using the new hash function $ h(k) = 3k\ mod\ 17 $.

In [14]:
def find_index(n):
    index = (3*n) % 13
    return index

# bucket array lst
lst = [None]*19     # N = 11 given in hashing function
for i in [54, 28, 41, 18, 10, 36, 25, 38, 12, 90]:
    index = find_index(i)
    if lst[index] is None:
        lst[index] = []     # secondary container for collision handling in separate chaining
    lst[index].append(i)
print(lst)

[None, None, [18], None, [10, 36], None, [54, 28, 41], None, None, None, [25, 38, 12, 90], None, None, None, None, None, None, None, None]


## R-10.15
Our `HashMapBase` class maintains a load factor $ \lambda \le 0.5 $. Reimplement that class to allow the user to specify the maximum load, and adjust the concrete subclasses accordingly.

In [15]:
from map_base import MapBase
from random import randrange

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression."""

    def __init__(self, load, cap=11, p=109345121):
        """Create an empty hash-table map."""
        self._table = cap*[None]
        self._n = 0
        self._prime = p
        self._scale = 1 + randrange(p-1)
        self._shift = randrange(p)
        self._load = load           # user input of load factor

    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    # This is part that was edited for R-10.15
    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) * self._load:     # maintain load factor
            self._resize(len(self._table)//self._load)
    
    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1
    
    def _resize(self, c):
        old = list(self.items())
        self._table = c*[None]
        self._n = 0
        for (k,v) in old:
            self[k] = v


## R-10.16
Give a pseudo-code description of an insertion into a hash table that uses quadratic probing to resolve collisions, assuming we also use the trick of replacing deleted entries with a special “deactivated entry” object.

Solution: similar to the problems we have done above.

## R-10.17
Modify our `ProbeHashMap` to use quadratic probing.

Solution: Change the last line of `_find_slot` method to incorporate quadratic probing.

## R-10.18
Explain why a hash table is not suited to implement a sorted map.

Solution: Element's index in hash map depend on hash function, not in any increasing or decreasing order.
While a sorted map need to store elements in incrasing or decreasing order.

## R-10.19
Describe how a sorted list implemented as a doubly linked list could be used to implement the sorted map ADT.

Solution: The best part about array-based list being used in sorted map is that we can apply binary search to find the exact location of a key if it is present, or the place where it should be if it is not present. It takes $ O(\log n) $ time to do binary search.  
If we use doubly linked list, we fortfeit the ability to fast search. If doubly linked list sorted then it will result in faster search, still worst case $ O(n) $

In [16]:
from map_base import MapBase

class SortedTableMap(MapBase):

    #-------------nonpublic behaviors-----------------
    def _find_position(self, k):
        position = self._table.first()
        while position is not None and position.element()._key < k:
            position = self._table.after(position)
        return position

    #-------------public behaviors-------------
    def __init__(self):
        self._table = PositionalList()      # Using PositionalList (doubly linked list)

    def __len__(self):
        return len(self._table)

    def __getitem__(self, k):
        p = self._find_position(k)
        if p is None or p.element()._key != k:
            raise KeyError('Key Error: ' + repr(k))
        return p.element()._value

    def __setitem__(self, k, v):
        p = self._find_position(k)
        if p is not None and p.element()._key == k:
            p.element()._value = v                   # reassign value
        elif len(self._table) == 0:
            self._table.add_first(self._Item(k,v))
        elif p is None:
            self._table.add_last(self._Item(k,v))
        else:
            self._table.add_before(p, self._Item(k,v))      # adds new item

    def __delitem__(self, k):
        p = self._find_position(k)
        if p is None or p.element()._key != k:
            raise KeyError('Key Error: ' + repr(k))
        self._table.delete(p)

    def __iter__(self):
        for item in self._table:
            yield item._key

    # def __reversed__(self):        # need to update PositionalList to support reversed
    #     for item in reversed(self._table):
    #         yield item._key
    
    def find_min(self):
        if len(self._table) > 0:
            return (self._table.first().element()._key, self._table.first().element()._value)
        else:
            return None

    def find_max(self):
        if len(self._table) > 0:
            return (self._table.last().element()._key, self._table.last().element()._value)
        else:
            return None


In [17]:
sortedmap = SortedTableMap()
for i in [('A', 1), ('B', 2), ('C', 3), ('D', 4), ('E', 5)]:
    sortedmap[i[0]] = i[1]

print(len(sortedmap))
for i in sortedmap:
    print(i, sortedmap[i])
print(sortedmap.find_min())
print(sortedmap.find_max())
del sortedmap['C']
for i in sortedmap:
    print(i, sortedmap[i])

# now check if insertion of random element is happening at proper place
sortedmap['C'] = 56
sortedmap['a'] = 10
sortedmap['z'] = 22
sortedmap['j'] = 88
print('\nafter random additions, check if order is maintained\n')
for i in sortedmap:
    print(i, sortedmap[i])      # working great; NOTE: A < a

print(f'max k,v pair: {sortedmap.find_max()}, min k,v pair: {sortedmap.find_min()}')

5
A 1
B 2
C 3
D 4
E 5
('A', 1)
('E', 5)
A 1
B 2
D 4
E 5

after random additions, check if order is maintained

A 1
B 2
C 56
D 4
E 5
a 10
j 88
z 22
max k,v pair: ('z', 22), min k,v pair: ('A', 1)


## R-10.20
What is the worst-case asymptotic running time for performing $ n $ deletions from a `SortedTableMap` instance that initially contains $ 2n $ entries?

Solution: If `SortedTableMap` is implemented with an array-based list (python list), then worst runtime for deletion is achieved when we delete the minimum element (popping first element) from the list every time.  

Deletion of single element happens in two steps:
1. Find the index of element by binary search. $ O(\log n) $
2. Popping the element at that index. $ O(n) $  

The dominant time taking activity in the given problem is described below:
1. Remove the first element from $ 2n $ elements. $ 2n-1 $ elements will be shifted left.
2. Remove the first element from now $ 2n-1 $ elements. $ 2n-2 $ elements will be shifted left.
3. This will continue until we remove first element from remaining $ n+1 $ elements. $ n $ elements will be shifted left.

So total time = $ (2n-1) + (2n-2) + (2n-3) +...+ (2n - n) $
= $ 2n^2 - (1+2+...+n) $
= $ \frac{3(n^2)-n}{2}  $

$ O(n^2) $

## R-10.21
Consider the following variant of the `_find_index` method from Code Fragment 10.8, in the context of the `SortedTableMap` class:
```
def _find_index(self, k, low, high):
    if high < low:
        return high + 1
    else:
        mid = (low + high) // 2
        if self._table[mid]._key < k:
            return self._find_index(k, mid + 1, high)
        else:
            return self._find_index(k, low, mid − 1)
```

Does this always produce the same result as the original version? Justify your answer.

Solution: Yes, it produces the same result.  
But it always takes $ \log n $ as we are running the algorithm until high < low. If we checked if mid key is equal to k, then we might have finished the algorithm sooner than $ \log n $.

## R-10.22
What is the expected running time of the methods for maintaining a maxima set if we insert $ n $ pairs such that each pair has lower cost and performance than one before it? What is contained in the sorted map at the end of this series of operations? What if each pair had a lower cost and higher performance than the one before it?

Solution: If each pair has lower cost and lower performance, then we are adding each pair at the beginning of sorted map(assuming we are using Python list for storage). It requires shifting all previous element to right, then placing the element at the start of map. It will never delete any element from the map as we don't have any pair with higher cost and lower performance. $ O(n^2) $  
At the end of series of such operations, we are left with $ n $ (cost, performance) pairs in the sorted map.  
If each pair had a lower cost and higher performance than one before it, then we will be left with just last (cost, performance) pair in the sorted map. Each addition will delete the one after it. $ O(n) $  

We have ignored the analysis of search operation each time a pair is being added because we use binary search which takes logarithmic time, and it will not be the dominant time in the entire operation.

## R-10.23
Draw an example skip list $ S $ that results from performing the following series of operations on the skip list shown in Figure 10.13: `del` $ S[38] $, $ S[48] = $'x' ,$ S[24] =$ 'y', `del` $ S[55] $. Record your coin flips, as well.

Solution: Refer the pdf imported from ipad.

## R-10.24
Give a pseudo-code description of the `__delitem__` map operation when using a skip list.

Solution: Refer the pdf imported from ipad.

## R-10.25
Give a concrete implementation of the `pop` method, in the context of a `MutableSet` abstract base class, that relies only on the five core set behaviors described in Section 10.5.2.

Solution:  The five core abstract methods of `MutableSet` are:  
1. `add`
2. `discard`
3. `__contains__`
4. `__len__`
5. `__iter__`

In [18]:
from collections.abc import MutableSet
from random import randrange

class Sets(MutableSet):
    
    def __init__(self):
        self._set = list()
    
    def add(self, e):
        if e not in self._set:
            self._set.append(e)
    
    def discard(self, e):
        if e in self._set:
            self._set.remove(e)
    
    def __contains__(self, e):
        return e in self._set
    
    def __len__(self):
        return len(self._set)
    
    def __iter__(self):
        for i in self._set:
            yield i

    def pop(self):
        if len(self._set) > 0:
            arbitary_element = self._set[randrange(len(self._set))]
            self.discard(arbitary_element)
            return arbitary_element
        else:
            raise KeyError
    
    def __str__(self):
        strng = ''
        for element in self._set:
            strng += f'{element}, '
        return strng


In [19]:
newset = Sets()
for i in range(10):     # add method working
    newset.add(i)
print(newset)           # __str__ method working

newset.discard(9)       # discard method working
print(newset)

print(10 in newset)
print(3 in newset)      # __contains__ method working

print(len(newset))      # __len__ method working

for i in newset:        # __iter__ method working
    print(i, end=', ')
print()

print(newset.pop())    # this should delete random elements in every execution, woking well


0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
0, 1, 2, 3, 4, 5, 6, 7, 8, 
False
True
9
0, 1, 2, 3, 4, 5, 6, 7, 8, 
7


## R-10.26
Give a concrete implementation of the `isdisjoint` method in the context of the `MutableSet` abstract base class, relying only on the five primary abstract methods of that class. Your algorithm should run in $ O(min(n, m)) $ where $ n $ and $ m $ denote the respective cardinalities of the two sets.

Solution: We will achieve $ O(min(n,m)) $ only when the set is implemented with hashing. Code written below doesn't use hashing, but the logic behind checking disjoint is same that has been asked in the problem.

In [20]:
class SetsAnother(Sets):

    def isdisjoint(self, other):
        n = len(self)
        m = len(other)
        if n < m:
            for e in self:          # iterate through min(n,m)
                other.add(e)        # each add will take O(1) if hashing is used
            return len(other) == n + m
        else:
            for e in other:
                self.add(e)
            return len(self) == n + m

In [21]:
set1 = SetsAnother()
for i in range(10):
    set1.add(i)

set2 = SetsAnother()
for i in range(9,20):
    set2.add(i)

print(set1.isdisjoint(set2))

False


## R-10.27
What abstraction would you use to manage a database of friends’ birthdays in order to support efficient queries such as “find all friends whose birthday is today” and “find the friend who will be the next to celebrate a birthday”?

Solution: We use `MultiMap`, where keys are birthdays. Each key is mapped to a container that contains the name of friends whose birthday is on a given date(key).